In [1]:
import random

Creating LLM Component

In [ ]:
class dummyLLM:
#Methods creation within class
    def __init__(self): #constrcutor method : default calling of class
        print("Dummy LLM Created")
    
    def predict(self, prompt):

        response_list  = [
            "Lucknow is capital of India",
            "Euro is european football championship",
            "LLM stands for large langauge models"
        ]
        return {'response':random.choice(response_list)}

In [47]:
llm = dummyLLM() # calling using constructor method

Dummy LLM Created


In [48]:
llm.predict("Who is ram?")

{'response': 'Lucknow is capital of India'}

Creating Template Component

In [49]:
class dummyLLMtemplate:

    def __init__(self,template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def format(self, input_dict):
        return self.template.format(**input_dict)

#** is dictionary unpacking

# It passes key–value pairs as named arguments

In [50]:
#giving values to  dummyLLMtemplate
template = dummyLLMtemplate(
    template= "Write a {length} story about {topic}",
    input_variables= ["length","topic"]
)

In [51]:
template

In [52]:
prompt = template.format({"length":"short",'topic':'india'}) #unpacking dict

In [53]:
prompt

'Write a short story about india'

In [54]:
llm = dummyLLM()

Dummy LLM Created


In [55]:
llm.predict(prompt)

{'response': 'LLM stands for large langauge models'}

Creating LLMChain

In [56]:
class LLMchain:
    def __init__(self,llm, prompt):
        self.llm = llm
        self.prompt = prompt

    def run(self,input_dict):

        final_prompt = self.prompt.format(input_dict)
        result = self.llm.predict(final_prompt)

        return result['response']
    
    # These chains are not flexible because it is hard to call this llm (run) more than one time like chain = prompt | llm | llm_output | llm
    # Because llm and prompt have diferent methods for calling or intercating

In [57]:
chain = LLMchain(llm,template)

In [58]:
chain.run({"length":"short",'topic':'india'})

'Lucknow is capital of India'

Standardized Components (LLM and Prompt) using runnables by having common methods

Using abstraction (OOP) (abstract class Runnable) : we will create all components which will inhertiate properties from runnable
and, so we will automatically to have implement same methods of Runnable class
in all of the components

In [59]:
from abc import ABC, abstractmethod

In [60]:
class Runnable(ABC):
    @abstractmethod  # it just abstract (no code for execution here)
    def invoke(input_data):
        pass

In [61]:
class dummyLLM(Runnable):
#Methods creation within class
    def __init__(self): #constrcutor method : deafult calling of class
        print("Dummy LLM Created")
    
    def predict(self, prompt):

        response_list  = [
            "Lucknow is capital of India",
            "Euro is european football championship",
            "LLM stands for large langauge models"
        ]
        return {'response':random.choice(response_list)}

In [62]:
# llm = dummyLLM()
# TypeError: Can't instantiate abstract class dummyLLM without an implementation for abstract method 'invoke'
# You have to use invoke method here : way of making sure

In [63]:
class dummyLLM(Runnable):
#Methods creation within class
    def __init__(self): #constrcutor method : deafult calling of class
        print("Dummy LLM Created")
    
     
    def predict(self, prompt):

        response_list  = [
            "Lucknow is capital of India",
            "Euro is european football championship",
            "LLM stands for large langauge models"
        ]
        return {'response':random.choice(response_list),"message" : "This method is going to be deprecated"}
    
    def invoke(self, prompt):

        response_list  = [
            "Lucknow is capital of India",
            "Euro is european football championship",
            "LLM stands for large langauge models"
        ]
        return {'response':random.choice(response_list)}
# We should not remove predict method because some people will be still using it
# need to give warning

In [64]:
class dummyLLMtemplate(Runnable):

    def __init__(self,template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def format(self, input_dict):
        return self.template.format(**input_dict + "This method is going to be deprecated")
    
    def invoke(self, input_dict):
        return self.template.format(**input_dict)

Forming chains

In [65]:
class RunnableConnector(Runnable):
    def __init__(self,runnable_list):
        self.runnable_list = runnable_list

    def invoke(self,input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
        return input_data


In [72]:
template = dummyLLMtemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [73]:
llm = dummyLLM()

Dummy LLM Created


In [74]:
chain = RunnableConnector([template,llm])

In [75]:
chain.invoke({"length":"short",'topic':'india'})

{'response': 'Lucknow is capital of India'}

Now, we can connect many components like parser, loader in as child of runnable abstract class and can connect easily

In [80]:
class parser(Runnable):

    def __init__(self):
      pass

    def invoke(self, input_data):
      return input_data['response']

In [81]:
parser = parser()

In [82]:
chain = RunnableConnector([template,llm,parser])

In [83]:
chain.invoke({"length":"short",'topic':'india'})

'LLM stands for large langauge models'